I'm only including this file out of the three, becuase they are all identical in code with the only differences being the edf file and the seizure times.
I'm not sure how much of the data you would like to see and Github to get ill when I tried to upload it all.

The three files are:
File 1 is patient 1
File 2 is patient 1 but 18 months later
File 3 is patient 2 with a VNS installed.

You can see all 3 files at the Githib: https://github.com/eipiisneg1/seizure_prediction_model.git

In [1]:
import mne
import warnings  # Hide all warnings here
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import mne
import warnings  # Hide all warnings here

mne.set_log_level('WARNING')
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
def load_pandas_data(filename):
    # When exporting data as a NumPy array or Pandas DataFrame, be sure to properly account for the unit of representation in your subsequent analyses.
    # with preload=True, contents are read into RAM

    mne.set_log_level('WARNING')
    warnings.simplefilter("ignore")

    patient = mne.io.read_raw(filename,preload=True)
    return patient.to_data_frame()


def get_seizure(seizure_set_df, start, end):
    """ return index locations for a given seizure set """

    start = np.where(seizure_set_df['time'] == start)
    end = np.where(seizure_set_df['time'] == end)
    seizure_set_df.loc[start[0][0]:end[0][0],'seizure = 1'] = 1
    return seizure_set_df


def prepare_df_for_analysis(preseizure_set, seizure_set):
    """ prepare, concate, and rename features for analysis """

    p1 = pd.concat([preseizure_set, seizure_set],axis=0,ignore_index=True)
    p1.set_index('time',inplace=True)

    # left hemisphere location totals
    p1['outer_left_hemi_sum'] = p1[['FP1-F7','F7-T7','T7-P7','P7-O1']].sum(axis=1)
    p1['inner_left_hemi_sum'] = p1[['FP1-F3','F3-C3','C3-P3','P3-O1']].sum(axis=1)

    # right hemisphere location totals
    p1['outter_right_hemi_sum'] = p1[['FP2-F8','F8-T8','T8-P8-0','P8-O2']].sum(axis=1)
    p1['inner_right_hemi_sum'] = p1[['FP2-F4','F4-C4','C4-P4','P4-O2']].sum(axis=1)

    # center totals
    p1['center_line'] = p1[['FZ-CZ','CZ-PZ']].sum(axis=1)

    # left temple to rear totals
    p1['left_temple_to_left_rear'] = p1[['FZ-CZ','CZ-PZ']].sum(axis=1)

    # temple to temple totals
    p1['temple_to_temple'] = p1[['FT9-FT10']].sum(axis=1)

    # left temple to rear totals
    p1['right_temple_to_right_rear'] = p1[['FT10-T8','T8-P8-1']].sum(axis=1)

    # reorder columns for better visual early analysis
    new_cols = ['seizure = 1',
                'FP2-F4','FP1-F7','F7-T7','T7-P7','P7-O1','outer_left_hemi_sum',
                'FP1-F3','F3-C3','C3-P3','P3-O1','inner_left_hemi_sum',
                'FP2-F8','F8-T8','T8-P8-0','P8-O2','outter_right_hemi_sum',
                'FP2-F4','F4-C4','C4-P4','P4-O2','inner_right_hemi_sum',
                'FZ-CZ','CZ-PZ','center_line',
                'P7-T7','T7-FT9','left_temple_to_left_rear',
                'FT9-FT10','temple_to_temple',
                'FT10-T8','T8-P8-1','right_temple_to_right_rear']

    p1 = p1.reindex(columns=new_cols)
    return p1


def is_df_clean(df):
    """ Check if a dataframe is clean. """
    # is_na = df.count()
    for i in range(df.count().all().tolist()):
        if not i:
            return f"There are null values in {df.columns[i]}: Note: this is expected if column is 'seizures = 1' as most of the values are zero, but 1 where the seizure exist."
    # infinite = np.isinf(df).sum()
    for j in  np.isinf(df).sum().any().to_list():
        if not j:
            return f'There are infinite values in {df.columns[j]}'

In [3]:
raw_p1_02_nonseizure = '/Users/jshensley/Desktop/springboard/DS_Method_and_Capstones/capstone_2_2/capstone_2/a_cookiecutter_for_capstone_2/{{ cookiecutter.seizure_prediction_model }}/pycharm_files/Capstone2_dataset/patient_1/intial_EEG/chb01_02_nonseizure_set_just_before_seizure_set.edf'

raw_p1_02_seizure_set = '/Users/jshensley/Desktop/springboard/DS_Method_and_Capstones/capstone_2_2/capstone_2/a_cookiecutter_for_capstone_2/{{ cookiecutter.seizure_prediction_model }}/pycharm_files/Capstone2_dataset/patient_1/intial_EEG/chb01_03_aa_seizure_set.edf'

p1_preseizure = load_pandas_data(raw_p1_02_nonseizure)
p1_preseizure['seizure = 1'] = 0

p1_seizure_set = load_pandas_data(raw_p1_02_seizure_set)
p1_seizure_set['seizure = 1'] = 0


p1_seizures = get_seizure(p1_seizure_set, 2996, 3036)
p1_df = prepare_df_for_analysis(p1_preseizure, p1_seizure_set)
p1_df_is_clean = is_df_clean(p1_df)

print(p1_df_is_clean)
print()

p1_df.info()
print(f'\n\n\n')

p1_describe = p1_df.describe()
print(p1_describe)


There are null values in seizure = 1: Note: this is expected if column is 'seizures = 1' as most of the values are zero, but 1 where the seizure exist.

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1843200 entries, 0.0 to 3599.99609375
Data columns (total 33 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   seizure = 1                 int64  
 1   FP2-F4                      float64
 2   FP1-F7                      float64
 3   F7-T7                       float64
 4   T7-P7                       float64
 5   P7-O1                       float64
 6   outer_left_hemi_sum         float64
 7   FP1-F3                      float64
 8   F3-C3                       float64
 9   C3-P3                       float64
 10  P3-O1                       float64
 11  inner_left_hemi_sum         float64
 12  FP2-F8                      float64
 13  F8-T8                       float64
 14  T8-P8-0                     float64
 15  P8-O2                 

In the columns below, the dash is actually a minus operator. 'FP2-F4' is the difference between the two leads in microvolts. This led me to include the sum columns with the brain locations for later use in the EDA visualization stage. I'm sure these will be removed for modeling.


![](../../../../../../../../Library/Messages/Attachments/75/05/9661CF27-B575-4CCA-949C-DC3888194CB9/IMG_4142.png)

In [4]:
p1_df.head()

,seizure = 1,FP2-F4,FP1-F7,F7-T7,T7-P7,P7-O1,outer_left_hemi_sum,FP1-F3,F3-C3,C3-P3,...,CZ-PZ,center_line,P7-T7,T7-FT9,left_temple_to_left_rear,FT9-FT10,temple_to_temple,FT10-T8,T8-P8-1,right_temple_to_right_rear
time,,,,,,,,,,,,,,,,,,,,,
0.000000,0,13.870574,15.042735,15.042735,28.327228,-13.479853,44.932845,40.830281,-12.307692,-12.307692,...,-26.373626,-58.998779,-27.936508,-47.863248,-58.998779,-25.592186,-25.592186,54.896215,-27.155067,27.741148
0.003906,0,0.195360,0.195360,0.195360,0.195360,0.195360,0.781441,0.195360,0.195360,0.195360,...,0.195360,0.390720,0.195360,0.195360,0.390720,0.195360,0.195360,0.195360,0.195360,0.390720
0.007812,0,0.195360,0.195360,0.195360,0.195360,0.195360,0.781441,0.195360,0.195360,0.195360,...,0.195360,0.390720,0.195360,0.195360,0.390720,0.195360,0.195360,0.195360,0.195360,0.390720
0.011719,0,-0.195360,-0.195360,0.195360,0.195360,0.195360,0.390720,-0.195360,0.195360,0.195360,...,0.195360,0.390720,0.195360,0.195360,0.390720,0.195360,0.195360,0.195360,0.195360,0.390720
0.015625,0,0.195360,-0.195360,0.976801,0.195360,0.195360,1.172161,0.195360,0.195360,-0.195360,...,0.195360,0.390720,0.195360,-1.758242,0.390720,0.586081,0.586081,0.586081,0.586081,1.172161


Additionally, I added the 'seizure = 1' column after you mentioned it. It also made it clear where the seizure was located at the outset for your review.

In [5]:
# The seizure
p1_df[p1_df['seizure = 1'] == 1]

,seizure = 1,FP2-F4,FP1-F7,F7-T7,T7-P7,P7-O1,outer_left_hemi_sum,FP1-F3,F3-C3,C3-P3,...,CZ-PZ,center_line,P7-T7,T7-FT9,left_temple_to_left_rear,FT9-FT10,temple_to_temple,FT10-T8,T8-P8-1,right_temple_to_right_rear
time,,,,,,,,,,,,,,,,,,,,,
2996.000000,1,10.354090,57.240537,45.909646,55.677656,3.321123,162.148962,-21.684982,42.393162,84.590965,...,113.894994,156.678877,-55.286935,18.949939,156.678877,24.810745,24.810745,6.446886,32.625153,39.072039
2996.003906,1,25.592186,53.333333,50.989011,56.459096,2.539683,163.321123,-19.731380,45.518926,88.888889,...,114.676435,150.427350,-56.068376,15.824176,150.427350,36.923077,36.923077,9.181929,21.294261,30.476190
2996.007812,1,38.876679,45.909646,59.194139,54.896215,-0.976801,159.023199,-23.247863,50.207570,88.888889,...,113.504274,139.096459,-54.505495,3.711844,139.096459,54.896215,54.896215,12.698413,10.744811,23.443223
2996.011719,1,45.518926,42.393162,64.273504,52.161172,-4.102564,154.725275,-22.075702,54.505495,83.028083,...,113.894994,134.017094,-51.770452,-6.446886,134.017094,68.962149,68.962149,15.824176,3.711844,19.536020
2996.015625,1,50.989011,46.300366,58.021978,46.691087,-2.539683,148.473748,-16.214896,53.333333,80.293040,...,115.457875,137.142857,-46.300366,2.930403,137.142857,73.650794,73.650794,12.698413,-5.665446,7.032967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035.984375,1,-36.923077,-0.976801,-28.717949,31.843712,14.652015,16.800977,-35.750916,-81.465201,85.763126,...,55.286935,40.244200,-31.452991,-13.089133,40.244200,92.405372,92.405372,23.247863,-16.996337,6.251526
3035.988281,1,-23.247863,15.824176,-29.499389,24.420024,6.446886,17.191697,-29.108669,-56.849817,74.432234,...,52.551893,35.946276,-24.029304,13.479853,35.946276,18.949939,18.949939,47.081807,-18.168498,28.913309
3035.992188,1,-30.671551,24.029304,-24.810745,12.307692,3.321123,14.847375,-35.750916,-42.002442,71.306471,...,47.863248,32.820513,-11.916972,18.168498,32.820513,24.420024,24.420024,-9.181929,-3.321123,-12.503053
